## Sqlite3 Tutorial
### Author: Jiawen Yan
### Date: 2018-04-23
### Version: 1.0

In [2]:
# by default, python3 has installed sqlite3, so you could simply import it 
import sqlite3

In [ ]:
# however, to actually browse database, you need to execute following commands in terminal:
'''
sudo add-apt-repository -y ppa:linuxgndu/sqlitebrowser
sudo apt-get update
sudo apt-get install sqlitebrowser
'''

### Part 1, Basics

In [23]:
# to connect to database, we need to establish a connection and cursor
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
#### your codes ####
c.close()
conn.close()

#### 1. Create a table in a database

In [24]:
'''
Datatypes: 
    NULL. The value is a NULL value.
    INTEGER. The value is a signed integer, stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.
    REAL. The value is a floating point value, stored as an 8-byte IEEE floating point number.
    TEXT. The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).
    BLOB. The value is a blob of data, stored exactly as it was input.
'''
def create_table(conn, c):
    c.execute("""CREATE TABLE IF NOT EXISTS Products(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                _id INTEGER,
                url TEXT,
                sku_id TEXT,
                domain TEXT,
                name TEXT,
                item_name TEXT,
                parameter1 TEXT,
                parameter2 TEXT, 
                price REAL ,
                brand TEXT ,
                other TEXT) ;""")
    conn.commit()
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
create_table(conn, c)
c.close()
conn.close()

#### 2. Insert Statement

In [25]:

def insert_data(conn, c, num):
    import pandas as pd
    fin = pd.read_csv("sample_jd_products.csv")
    for i in range(1, num+1):
        c.execute('''INSERT INTO Products 
        (_id, url, sku_id, domain, name, item_name, parameter1, parameter2, price, brand, other) 
        VALUES ("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}") ; '''.format( *[ fin.loc[i][j] for j in range(1,12) ]) )
        conn.commit()
    

conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
insert_data(conn, c, 2000)
c.close()
conn.close()


#### 3. Select Statement

In [5]:
def select_data(conn, c, Id):
    ret = c.execute("SELECT sku_id, name, price FROM Products WHERE id = {}".format(Id))
    ret = ret.fetchall()
    if ret:
        print(ret[0])
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
select_data(conn, c, Id = 128)
c.close()
conn.close()


('1718026732', "['凯伊秀眼霜   能量紧致明亮眼霜15g  女去黑眼圈眼袋细纹淡化鱼尾纹']", '109.0')


#### 4. Update statement

In [8]:
def update_data(conn, c, Id, new_price):
    ret = c.execute("UPDATE Products SET price = {} WHERE id = {}".format(new_price, Id))

conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
select_data(conn, c, Id = 128)
update_data(conn, c, Id = 128, new_price=12.0)
select_data(conn, c, Id = 128)
c.close()
conn.close()


('1718026732', "['凯伊秀眼霜   能量紧致明亮眼霜15g  女去黑眼圈眼袋细纹淡化鱼尾纹']", '109.0')
('1718026732', "['凯伊秀眼霜   能量紧致明亮眼霜15g  女去黑眼圈眼袋细纹淡化鱼尾纹']", '12.0')


#### 5. Delete Statement 

In [26]:
def delete_data(conn, c, Id):
    c.execute("DELETE FROM Products WHERE id = {} ;".format(Id) )
    
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

select_data(conn, c, Id = 999)
delete_data(conn, c, Id = 999)
conn.commit()
select_data(conn, c, Id = 999)

c.close()
conn.close()


#### *6. Time efficience

In [4]:
# how to quickly select data from database?
from datetime import datetime
import random

def many_selects(conn, c, times):
    sku_ids = ["1106257762","1055818589","1434612168","1339190822","1050895058",
               "1077734754","1391081652","1101196302","1435256636","1222981002"]
    time_start = datetime.now()
    for i in range(times):
        Id = random.choice(sku_ids)
        ret = c.execute("SELECT sku_id, name, price FROM Products WHERE sku_id = {}".format(Id))
    time_end = datetime.now()
    print(time_end-time_start)

conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
many_selects(conn, c, 200000)
c.close()
conn.close()


0:00:10.874353


In [14]:
# trick: create index
def remove_index(conn, c):
    c.execute("DROP INDEX sku_id_index ;")
    conn.commit()
def create_index(conn, c):
    c.execute("CREATE UNIQUE INDEX IF NOT EXISTS sku_id_index ON Products (sku_id);")
    conn.commit()
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
create_index(conn, c)
#remove_index(conn, c)
c.close()
conn.close()


In [15]:
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
many_selects(conn, c, 200000)
c.close()
conn.close()

0:00:05.495829


### Part2, Intermediate 

#### 1. Select Order and Limit

In [ ]:
'''
SQL SYNTAX:
SELECT column-names
  FROM table-name
 ORDER BY column-names
LIMIT X;
'''

In [33]:
# select ... order by statement
# select ... limit 5 
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
select_data_order(conn, c)
ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price > 1000.0 ORDER BY price ASC LIMIT 10; ")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
c.close()
conn.close()

('6100165', "SK-II'小灯泡'曜白精华套装（肌因光蕴环采钻白精华露30ml+护肤精华露10ml）（面部精华液）", 1040.0)
('4380118', "SK-II'小灯泡'曜白精华套装（肌因光蕴环采钻白精华露30ml+护肤精华露10ml）（面部精华液）", 1040.0)
('2223524', 'SK-II护肤面膜10p（日本面膜女 补水保湿）', 1060.0)
('5981507', '玉兰油Olay新生塑颜臻粹化妆品套装(小脸精华+小脸霜+洁面+嫩肤水+面膜+走珠笔+黄金棒+手包)', 1090.0)
('1452961433', '圣荷（ST.HERB） 泰国圣荷纳诺美胸霜丰胸产品美乳精油霜15ml/支 胸部护理霜贴 1月装-30天量', 1160.0)
('5203591', '泰国圣荷（ST.HERB）纳米美胸增大一月超值套装 圣荷美胸霜丰胸产品 胸部护理按摩精油霜', 1160.0)
('1717795340', "['佰草童话  魔法精油套']", 1176.0)
('6100187', "SK-II PITERA'神仙水'+'大眼眼霜' 紧致套装（护肤精华露75ml+微肌因修护眼霜15g）（面部精华液 补水保湿）", 1180.0)
('1366031', '后Whoo 津率享红华凝香平颜系列礼盒6件套315ml（又名：津率享红华凝香护肤品礼盒6件套 新老包装随机发货）', 1210.0)
('2562502', '后Whoo 拱辰享气韵生礼盒7件套336ml（乳液+滋养霜+眼霜+唇膏）补水保湿 乳液 面霜 眼霜 唇膏 套装 拱辰享', 1220.0)


#### 2. Select Offset top X rows

In [35]:
# Offset top X rows
    
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price > 1000.0 ORDER BY price ASC LIMIT 10 OFFSET 10; ")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
c.close()
conn.close()

('1032200369', '圣荷（ST.HERB） 泰国圣荷美胸霜丰胸霜丰乳精油产品美乳霜贴露膏 胸部增大1月加强装', 1299.0)
('234261', '希思黎（Sisley）全能乳液125ml（乳液 补水保湿滋养 护肤品）(新老包装随机发放）', 1299.0)
('5878510', "SK-II限量版'神仙水'改变瓶晶透修护礼盒（神仙水230ml+洁面霜20g+唯白面膜2p+环采亮霜0.8g+环采霜2gx2）", 1370.0)
('6100177', "SK-II PITERA'神仙水'男士醒肤净透清爽护肤套装（男士护肤精华露230ml）（面部精华露 男控油保湿补水）", 1370.0)
('6088552', "SK-II PITERA'神仙水'晶透修护肌肤套装（护肤精华露230ml）（面部精华液 补水保湿 紧致修复 ）", 1370.0)
('5878494', "SK-II限量版'神仙水'宣言瓶晶透修护礼盒（神仙水230ml+洁面霜20g+唯白面膜2p+环采亮霜0.8g+环采霜2gx2）", 1370.0)
('2574022', "SK-II男士焕活护肤精华露230ml（PITERA'神仙水' 面部精华露 男士控油保湿补水 紧致修复 ）", 1370.0)
('16676298276', '德沃思（DEWOS） 骨盆修复仪髋骨调理仪盆骨矫正仪器修复仪器产后理疗卵巢保养 髋骨调理仪', 1380.0)
('6100191', 'SK-II微肌因赋活修护套装（微肌因修护精华霜50g+微肌因修护眼霜15g）（乳液面霜补水保湿 淡化细纹去眼圈）', 1450.0)
('1526049919', '颜如玉 胶原低聚肽蓝莓口服液胶原蛋白口服蓝莓果饮国皂 颜如玉12盒装', 1500.0)


#### 3.Select Distinct

In [ ]:
'''
SQL SYNTAX:
SELECT DISTINCT column-name
  FROM table-name
'''

In [39]:
# distinct select, to get unique value(s)
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

ret = c.execute("SELECT DISTINCT name, price FROM Products WHERE price > 1000.0 ORDER BY price ASC LIMIT 10; ")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)

c.close()
conn.close()

("SK-II'小灯泡'曜白精华套装（肌因光蕴环采钻白精华露30ml+护肤精华露10ml）（面部精华液）", 1040.0)
('SK-II护肤面膜10p（日本面膜女 补水保湿）', 1060.0)
('玉兰油Olay新生塑颜臻粹化妆品套装(小脸精华+小脸霜+洁面+嫩肤水+面膜+走珠笔+黄金棒+手包)', 1090.0)
('圣荷（ST.HERB） 泰国圣荷纳诺美胸霜丰胸产品美乳精油霜15ml/支 胸部护理霜贴 1月装-30天量', 1160.0)
('泰国圣荷（ST.HERB）纳米美胸增大一月超值套装 圣荷美胸霜丰胸产品 胸部护理按摩精油霜', 1160.0)
("['佰草童话  魔法精油套']", 1176.0)
("SK-II PITERA'神仙水'+'大眼眼霜' 紧致套装（护肤精华露75ml+微肌因修护眼霜15g）（面部精华液 补水保湿）", 1180.0)
('后Whoo 津率享红华凝香平颜系列礼盒6件套315ml（又名：津率享红华凝香护肤品礼盒6件套 新老包装随机发货）', 1210.0)
('后Whoo 拱辰享气韵生礼盒7件套336ml（乳液+滋养霜+眼霜+唇膏）补水保湿 乳液 面霜 眼霜 唇膏 套装 拱辰享', 1220.0)
('圣荷（ST.HERB） 泰国圣荷美胸霜丰胸霜丰乳精油产品美乳霜贴露膏 胸部增大1月加强装', 1299.0)


#### 4. Select MIN/MAX 

In [ ]:
'''
SQL SYNTAX:
SELECT MIN/MAX(column-name)
  FROM table-name
'''

In [ ]:
# select MIN/MAX

conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

ret = c.execute("SELECT MAX (price) FROM Products WHERE price < 9000")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
        
c.close()
conn.close()

#### 5. Select COUNT/SUM/AVG

In [ ]:
'''
SQL SYNTAX:
SELECT COUNT/SUM/AVG(column-name)
  FROM table-name
'''

In [46]:
# Count 
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT COUNT (sku_id) FROM Products")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
c.close()
conn.close()

(2000,)


In [47]:
# Count 
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT SUM (price) FROM Products")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
c.close()
conn.close()

(312363.20000000135,)


In [48]:
# AVG 
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT AVG (price) FROM Products")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
c.close()
conn.close()

(156.18160000000069,)


### Part3, Conditions and Logics

#### 1. And/Or/Not logics

In [8]:
# AND
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price < 1000 AND price > 900")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
        
c.close()
conn.close()

('6100179', "SK-II'小银瓶'耀白无瑕套装（肌因光蕴精华露30ml+护肤精华露10ml）（面部精华液）", 960.0)
('1304764', '后Whoo 拱辰享气韵生润颜系列礼盒6件套350ml（保湿 滋润 护肤礼盒 拱辰享基础套装）', 958.0)


In [3]:
# NOT
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

ret = c.execute("SELECT sku_id, name, price FROM Products WHERE NOT price <1000 ")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
        break
        
c.close()
conn.close()

('2562502', '后Whoo 拱辰享气韵生礼盒7件套336ml（乳液+滋养霜+眼霜+唇膏）补水保湿 乳液 面霜 眼霜 唇膏 套装 拱辰享', 1220.0)


#### 2. Between


In [11]:
# NOT
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()

ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price BETWEEN 900 AND 1000 ")
ret = ret.fetchall()
if ret:
    for r in ret:
        print(r)
        
c.close()
conn.close()

('6100179', "SK-II'小银瓶'耀白无瑕套装（肌因光蕴精华露30ml+护肤精华露10ml）（面部精华液）", 960.0)
('1304764', '后Whoo 拱辰享气韵生润颜系列礼盒6件套350ml（保湿 滋润 护肤礼盒 拱辰享基础套装）', 958.0)


#### 3.time efficiency


In [15]:
from datetime import datetime
# Between ... and
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
st = datetime.now()
for _ in range(20000):
    ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price BETWEEN 900 AND 1000 ")
    ret = ret.fetchall()
et = datetime.now()
print(et-st)
c.close()
conn.close()
# AND 
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
st = datetime.now()
for _ in range(20000):
    ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price >= 900 AND price <=1000 ")
    ret = ret.fetchall()
et = datetime.now()
print(et-st)
c.close()
conn.close()

0:00:06.378568
0:00:06.283135


#### 4.  In statement 

In [24]:
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT sku_id, name, price FROM Products WHERE price IN ('100');")
ret = ret.fetchall()
print(ret)
c.close()
conn.close()

[('1951395', '欧珀莱（AUPRES）均衡保湿系列 柔润水（滋润型） 150ml（清透补水 持久保湿）', 100.0), ('1463206373', "['雪雅璐御发诗洗发水洗发露水光滋养无硅轻盈焗油修护 水光滋养洗发水+焗油精华护发乳']", 100.0), ('1463206374', "['雪雅璐御发诗洗发水洗发露水光滋养无硅轻盈焗油修护 无硅轻盈洗发水+焗油精华护发乳']", 100.0)]


In [3]:
conn = sqlite3.connect("sqlite3_tutorial_JD_data.db")
c = conn.cursor()
ret = c.execute("SELECT sku_id, name, price FROM Products WHERE name LIKE '%SK-II%';")
ret = ret.fetchall()
for r in ret: 
    print(r)
    break
c.close()
conn.close()

('3342868', 'SK-II嫩肤清莹露160ml（爽肤水 女 保湿补水 柔肤水  深层清洁）', 560.0)
